# Interactive maps with the folium package


In [ ]:
import pandas as pd
import geopandas as gpd
import math

In [ ]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

# Interactive Map

### A relatively simple map with folium.Map()

In [ ]:
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)
m_1

# Several arguments customize the appearance of the map:

```location``` sets the initial center of the map. We use the latitude (42.32° N) and longitude (-71.0589° E) of the city of Boston.

```tiles``` changes the styling of the map; in this case, we choose the OpenStreetMap style. If you're curious, you can find the other options listed here.

cartodbdark_matter

cartodbpositron

cartodbpositronnolabels

cartodbpositrononlylabels

openstreetmap

stamenterrain

stamentoner

stamentonerbackground

stamentonerlabels

stamenwatercolor

```zoom_start``` sets the initial level of zoom of the map, where higher values zoom in closer to the map.
Take the time now to explore by zooming in and out, or by dragging the map in different directions.

# Read Crimes in Boston Dataset

In [ ]:
crimes = pd.read_csv("../input/crimes-in-boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()

To reduce the amount of data we need to fit on the map, we'll (temporarily) confine our attention to daytime robberies.

In [ ]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(7,23))))]

# folium.Marker
## folium.Marker()
We add markers to the map with folium.Marker(). Each marker below corresponds to a different robbery.

In [ ]:
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)
m_2

# Cluster lots of data
## folium.plugins.MarkerCluster¶

In [ ]:
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

# Display the map
m_3


# Buble Map
## folium.Circle()

In [ ]:
m_4 = folium.Map(location=[42.32,-71.0589], tiles="cartodbpositron",zoom_start=15)
def color_val(val):
    return 'forestgreen' if (val<=12) else "darkred"
    
for i in range(0,len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=40,
        color=color_val(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)
m_4

```location``` is a list containing the center of the circle, in latitude and longitude.

```radius``` sets the radius of the circle.
Note that in a traditional bubble map, the radius of each circle is allowed to vary. We can implement this by defining a function similar to the ```color_producer()``` function that is used to vary the color of each circle.

```color``` sets the color of each circle.

The ```color_producer()``` function is used to visualize the effect of the hour on robbery location.

# Heatmap
## folium.plugin.Heatmap()

In [ ]:
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_5)
m_5

```data``` is a DataFrame containing the locations that we'd like to plot.

```radius``` controls the smoothness of the heatmap. Higher values make the heatmap look smoother (i.e., with fewer gaps).

# Choropleth maps

In [ ]:
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('../input/boston-police-districts/Police_Districts.shp')
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

In [ ]:
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

In [ ]:
m_6 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

Choropleth(geo_data=districts.__geo_interface__, data=plot_dict, key_on="feature.id", fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)').add_to(m_6)
m_6

```geo_data``` is a GeoJSON FeatureCollection containing the boundaries of each geographical area.

In the code above, we convert the districts GeoDataFrame to a GeoJSON FeatureCollection with the __geo_interface__ attribute.
```data``` is a Pandas Series containing the values that will be used to color-code each geographical area.

```key_on``` will always be set to feature.id.

This refers to the fact that the GeoDataFrame used for geo_data and the Pandas Series provided in data have the same index. To understand the details, we'd have to look more closely at the structure of a GeoJSON Feature Collection (where the value corresponding to the "features" key is a list, wherein each entry is a dictionary containing an "id" key).

```fill_color``` sets the color scale.

```legend_name``` labels the legend in the top right corner of the map.